In [1]:
import pandas as pd
import os
import datetime
import numpy as np
import time
import smtplib
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
import dropbox
from PIL import Image 
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import *
client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')

## Get List of Files

In [5]:
def files():
    client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')
    metadata = client.metadata('/')
    
    files = [content['path'].split('/')[-1] for content in metadata['contents'] if content['is_dir'] == False]
    size = [round(content['bytes']/10**6, 2) for content in metadata['contents'] if content['is_dir'] == False]
    dates = [content['modified'] for content in metadata['contents'] if content['is_dir'] == False]
    
    name = [i[0:i.find('_')].lower().title() for i in files]
    files = pd.DataFrame({'file_name': files, 'c_name': name, 'size': size, 'dates': dates})
    
    files['date_mod_d'] = [time.strptime(i, "%a, %d %b %Y %H:%M:%S %z") for i in dates]
    files['date_mod_d'] = [datetime.datetime(*files.date_mod_d.loc[i][:6]) - datetime.timedelta(hours = 4) for i in files.index]
    files['date_mod'] = [files.loc[i].date_mod_d.date() for i in files.index]
    files['hour_mod'] = [files.loc[i].date_mod_d.time() for i in files.index]
    #tday = datetime.datetime.today().date()
    #files = files[files.date_mod == tday]
    
    del files['date_mod_d'], files['dates']
    return files   

In [5]:
files().head()

,c_name,file_name,size,date_mod,hour_mod
0,ABA DUENAS,ABA DUENAS_0001.JPG,0.12,2017-05-18,10:44:25
1,ABA DUENAS,ABA DUENAS_0005.JPG,0.12,2017-05-17,19:26:31
2,ABA DUENAS,ABA DUENAS_0009.JPG,0.12,2017-05-17,19:26:31
3,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0001.JPG,0.21,2017-05-12,16:03:13
4,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0013.JPG,0.07,2017-05-12,16:03:15


## Get List of Today's Clients (User Input).

In [10]:
def clients():
    clients = pd.read_excel('Registro de Clientes.xlsx')
    return clients

## Final List With Emails.

In [11]:
def sendPic():
    sendPic = pd.merge(files(), clients, left_on = 'c_name', right_on = 'Nombre')
    return sendPic


## Time related Functionalities.

In [ ]:
#def time():
#    exit = datetime.datetime.now()
#    td = datetime.timedelta(minutes = 30)
#    while datetime.datetime.now() < 
#        print('early')
        
#    return datetime.datetime.now().hour
#time()

## Email Sending Functionalities

In [24]:
def SendMail4(send_list, files):
    
    msg = MIMEMultipart()
    msg['Subject'] = 'prueba'
    msg['From'] = 'fapb88ve@gmail.com'
    msg['To'] = 'fapb88ve@gmail.com'
    text = MIMEText("test")
    msg.attach(text)
    for i in files:
        if 'mp4' in i:
            part = MIMEBase('application', "octet-stream")
            fo=open('ADRIANA GRAELL WK27_0013.mpg',"rb")
            part.set_payload(fo.read() )
            Encoders.encode_base64(part)
            part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename('ADRIANA GRAELL WK27_0013.mpg'))
            msg.attach(part)
        if 'jpg' in i:
            img_data = open(k, 'rb').read()
            image = MIMEImage(img_data, name=os.path.basename(k))
            msg.attach(image)
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.ehlo()
    s.starttls()
    s.login('fapb88ve@gmail.com', )
    s.sendmail('fapb88ve@gmail.com', 'fapb88ve@gmail.com', msg.as_string())
    s.quit()


## Watermarking

In [3]:
def watermark(x):
    for i in x:
        if 'AVI' in i:
            clip = VideoFileClip(i)
            #logo = (ImageClip("pancita4.png")
              #.set_duration(clip.duration)
           #   .resize(height=80)  
            #  .margin(right=8, top=8, opacity=0) # (optional) logo-border padding
             # .set_pos(("right","bottom")))
            final = CompositeVideoClip([clip, logo])
            final.write_videofile("Pancitas " + i +".mp4")
        if 'JPG' in i:
            #plogo = Image.open("pancita4.png")
            #plogow, plogoh = plogo.size
            #plogo = plogo.resize((int(plogow*.1), int(plogoh*.1)))
            #plogow, plogoh = plogo.size
            pic = Image.open(i)
            picw, pich = pic.size
            pic.paste(plogo, (picw - plogow, pich - plogoh), plogo)
            pic.save('Pancitas '+ i)
            
        else:
            pass


## Download and upload files

In [13]:
def dl_files(x):
    df = pd.DataFrame(x.groupby('Nombre').Nombre.count())
    files = x.file_name
    for i in df.index:
        c_dir = 'Pancitas Ultrasound {}'.format(i.lower().title())
        os.mkdir(c_dir)
        os.chdir('.\\' + c_dir)
        dl_df = []
        for k in files:
            if i.upper() in k:
                dl_df.append(k)
        for j in dl_df:
            f, metadata = client.get_file_and_metadata('/'+j)
            out = open(j, 'wb')
            out.write(f.read())
            out.close()
        watermark(dl_df)
        for l in os.listdir(c_dir):
            if 'Pancitas' in l:
                pass
            else:
                os.remove(l)
        os.chdir('..\\') 

## Body Function

In [15]:
def body():
    files = files()
    clients = pd.read_excel('Registro de Clientes.xlsx')
    send_pic = pd.merge(files(), clients, left_on = 'c_name', right_on = 'Nombre')
    dl_files(send_pic)
    watermark(send_f)

## Code implemented

In [7]:
def full_body():
    client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')
    print('Coloque su correo personal. Todo correo que no pueda ser enviado, sera reenviado al ingresado.')
    sup_email = input()
    plogo = Image.open("pancita4.png")
    plogow, plogoh = plogo.size
    plogo = plogo.resize((int(plogow*.1), int(plogoh*.1)))
    plogow, plogoh = plogo.size
    logo = (ImageClip("pancita4.png")
              .set_duration(clip.duration)
              .resize(height=80) # if you need to resize...
              .margin(right=8, top=8, opacity=0) # (optional) logo-border padding
              .set_pos(("right","bottom")))
    while datetime.datetime.now().time() <= datetime.datetime.strptime('19:45', '%H:%M'):
        if datetime.datetime.now().time() == datetime.datetime.strptime('12:30', '%H:%M'):
            files = files()
            clients_tday = clients()
            send_f = sendPic()
            dl_files(send_f.file_name)
            watermark(send_f)
        
        
      



In [5]:
clients = pd.read_excel('Registro de Clientes.xlsx')

In [9]:
send_pic = pd.merge(files(), clients, left_on = 'c_name', right_on = 'Nombre')

In [10]:
#dl_files(files())

In [15]:
def dl_files(x):
    df = pd.DataFrame(x.groupby('Nombre').Nombre.count())
    files = x.file_name
    for i in df.index:
        c_dir = 'Pancitas Ultrasound {}'.format(i.lower().title())
        os.mkdir(c_dir)
        os.chdir('.\\' + c_dir)
        dl_df = []
        for k in files:
            if i.upper() in k:
                dl_df.append(k)
        for j in dl_df:
            f, metadata = client.get_file_and_metadata('/'+j)
            out = open(j, 'wb')
            out.write(f.read())
            out.close()
        os.chdir('..\\') 
dl_files(send_pic)
os.getcwd()

'C:\\Users\\Frank Pinto\\desktop\\pds\\Pancitas\\Pancitas Ultrasound Aba Duenas'